# VISTA-CA EDA + Estimating CH$_4$ from Cows
## ER 131 Project | Group 4
**Author: ** ['Marshall Worsham'] <br>
**Date: ** 10-29-2020

## Motivations
* Enteric fermentation from cattle is a major source of $CH_4$, a greenhouse gas with nearly two orders of magnitude greater warming potential than $CO_2$
* Because of the difficulty of directly measuring enteric emissions in situ, large-scale emissions estimates are typically generated from mathematical models calculated at the individual and scaled up to population from agricultural census data
* The amount of $CH_4$ produced from enteric fermentation is primarily a function of intake (feed) mass and feed composition
* From the literature there are a handful of models available to calculate $CH_4$ emissions: 
    * Moraes et al. (2014)
    * IPCC (2006)
    * Nielsen et al. (2013)
    * Appuhamy et al. (2018)
    * US EPA

## Formulas for Estimating CH$_4$ Emissions (g/cow/d) 

### Dairy
#### IPCC
$CH_{4|dairy} = \frac{0.065 * GEI}{0.05565}$ <br>
#### US EPA 
$CH_{4|dairy} = \frac{(0.059 * GEI)}{0.05565}$ <br>
#### Moraes 
$CH_{4|dairy} = \frac{2.88 + 0.053 * GEI - 0.190 EE}{0.05565}$ <br>
#### Appuhamy
$CH_{4|dairy} = CH_{4|lactating} + CH_{4|dry} + CH_{4|replacement}$ <br>
* $CH_{4|lactating} = 11.2*DMI + 2.18*dNDF + 32.2*Milk fat$
* $CH_{4|dry} = 9.6 + 22.1 *DMI$
* $CH_{4|replacement} = 9.6 + 22.1 * DMI$

DMI = dry matter intake (kg/d)<br>
GEI = gross energy intake (MJ/d) <br>
EE = ether extract (% dry matter) <br>
dNDF = digestible neutral detergent fiber (% dry matter) <br>


## Libraries and dependencies

In [140]:
import pandas as pd   
import os
import numpy as np
import zipfile
import geopandas as gpd
import matplotlib.pyplot as plt
import netCDF4 as nc

## California Cow Intake Data
(Appuhamy et al. 2018)

In [2]:
# make a dataframe with mean values of GMI, GEI, dNDF, EE, Milk fat for CA cows, reported in Appuhamy et al. 2018
intake = pd.DataFrame(
    [[13.5, 248.2, 45.9, 3.6, 0], [9.9, 181.2, 47.9, 3.2, 0], [19.9, 381.0, 34.1, 3.2, 3.6], [10.3, 173.0, 45.8, 3.4, 0.0]],
    columns = ['DMI_mean', 'GEI_mean', 'dNDF_mean', 'EE_mean', 'Milk Fat'])
intake['type'] = ['CA Heifer', 'CA Dry Cow', 'CA Lactating Cow', 'Generic estimate from literature']

In [3]:
intake

,DMI_mean,GEI_mean,dNDF_mean,EE_mean,Milk Fat,type
0,13.5,248.2,45.9,3.6,0.0,CA Heifer
1,9.9,181.2,47.9,3.2,0.0,CA Dry Cow
2,19.9,381.0,34.1,3.2,3.6,CA Lactating Cow
3,10.3,173.0,45.8,3.4,0.0,Generic estimate from literature


## VISTA-CA
Hopkins et al. 2019 provide point shapefiles of known and expected point sources of methane emissions in California as part of the North American Carbon Program. Below we load and plot the locations of 1787 dairy and cattle feed lot operations in CA.

#### Citation
Marklein, A.R., D. Meyer, M.L. Fischer, S. Jeong, T. Rafiq, M. Carr, and F.M. Hopkins. 2020. Methane Emissions from Dairy Sources (Vista-CA), State of California, USA, 2019. ORNL DAAC, Oak Ridge, Tennessee, USA. [https://doi.org/10.3334/ORNLDAAC/1814](https://doi.org/10.3334/ORNLDAAC/1814)

In [3]:
# unzip files from the repository
# these are in the Cows.zip package
# this should load them, as long as the files remain in the same directory as this notebook

vista = 'VISTA'
os.listdir(vista)
z = os.listdir(vista)[1]
zf = zipfile.ZipFile(os.sep.join(['.', vista, z]), 'r')
zf.printdir()
fl = zf.extractall()
[i for i in zf.namelist() if 'shp' in i]

File Name                                             Modified             Size
Vista_CA_Dairies/Vista_CA_Dairies.dbf          2019-11-10 10:36:00      3130229
Vista_CA_Dairies/Vista_CA_Dairies.prj          2019-10-02 13:13:18          145
Vista_CA_Dairies/Vista_CA_Dairies.sbn          2019-11-05 17:33:36        16092
Vista_CA_Dairies/Vista_CA_Dairies.sbx          2019-11-05 17:33:36          540
Vista_CA_Dairies/Vista_CA_Dairies.shp          2019-11-10 10:36:00        75560
Vista_CA_Dairies/Vista_CA_Dairies.shx          2019-11-10 10:36:00        13820


['Vista_CA_Dairies/Vista_CA_Dairies.shp']

In [4]:
# open the files and store data in a geodataframe

vista_ch4 = gpd.GeoDataFrame() # initialize empty dataframe

vista = 'VISTA'
for z in os.listdir(vista):
    zf = zipfile.ZipFile(os.sep.join(['.', vista, z]), 'r')
    zf.printdir()
    shapes = zf.extractall()
    sf = [i for i in zf.namelist() if 'shp' in i]
    gpdf = gpd.read_file(sf[0], low_memory = False)
    vista_ch4 = vista_ch4.append(gpdf)

File Name                                             Modified             Size
Vista_CA_Feed_Lots/Vista_CA_Feed_Lots.dbf      2019-11-10 10:34:08       131754
Vista_CA_Feed_Lots/Vista_CA_Feed_Lots.prj      2019-07-09 22:07:12          145
Vista_CA_Feed_Lots/Vista_CA_Feed_Lots.sbn      2019-11-05 17:41:16          836
Vista_CA_Feed_Lots/Vista_CA_Feed_Lots.sbx      2019-11-05 17:41:16          148
Vista_CA_Feed_Lots/Vista_CA_Feed_Lots.shp      2019-11-10 10:34:08         3268
Vista_CA_Feed_Lots/Vista_CA_Feed_Lots.shx      2019-11-10 10:34:08          676
File Name                                             Modified             Size
Vista_CA_Dairies/Vista_CA_Dairies.dbf          2019-11-10 10:36:00      3130229
Vista_CA_Dairies/Vista_CA_Dairies.prj          2019-10-02 13:13:18          145
Vista_CA_Dairies/Vista_CA_Dairies.sbn          2019-11-05 17:33:36        16092
Vista_CA_Dairies/Vista_CA_Dairies.sbx          2019-11-05 17:33:36          540
Vista_CA_Dairies/Vista_CA_Dairies.shp   

In [5]:
# check the shape and head of the gpdf
print(vista_ch4.shape)
vista_ch4.head()

(1787, 11)


,Latitude,Longitude,City,State,Source,Vista_ID,VistaIPCC,VistaName,VistaSType,VistaDate,geometry
0,37.519455,-120.749914,DENAIR,CA,RAFIQ,FDL000022,3A1 Enteric Fermentation,G & S Cattle Co of Turlock,Feed Lot,2019-11-10,POINT Z (-120.74991 37.51946 0.00000)
1,37.727375,-120.841383,OAKDALE,CA,RAFIQ,FDL000019,3A1 Enteric Fermentation,Farmers Livestock Market,Feed Lot,2019-11-10,POINT Z (-120.84138 37.72738 0.00000)
2,38.304731,-121.316172,GALT,CA,RAFIQ,FDL000007,3A1 Enteric Fermentation,Cattlemen's Livestock Market,Feed Lot,2019-11-10,POINT Z (-121.31617 38.30473 0.00000)
3,38.328705,-121.188084,HERALD,CA,RAFIQ,FDL000010,3A1 Enteric Fermentation,Clay Station Feed Lot,Feed Lot,2019-11-10,POINT Z (-121.18808 38.32871 0.00000)
4,39.698616,-122.199166,ORLAND,CA,RAFIQ,FDL000045,3A1 Enteric Fermentation,Orland Livestock Commission,Feed Lot,2019-11-10,POINT Z (-122.19917 39.69862 0.00000)


### Import CA counties to use as a base map 

In [8]:
counties = gpd.read_file(os.sep.join(['.', 'CA_Counties', 'CA_Counties_TIGER2016.shp']))
counties.plot()

ImportError: The descartes package is required for plotting polygons in geopandas. You can install it using 'conda install -c conda-forge descartes' or 'pip install descartes'.

In [9]:
# match CRS
print('counties: ', counties.crs)
print('ch4: ', vista_ch4.crs)
counties_wgs84 = counties.to_crs(epsg=4326)
assert counties_wgs84.crs == vista_ch4.crs == 'epsg:4326'

counties:  epsg:3857
ch4:  epsg:4326


In [11]:
fig, ax = plt.subplots(figsize=(10,20))
counties_wgs84.plot(ax=ax, color = 'black', edgecolor='white', alpha = 0.8)
vista_ch4.plot(ax=ax, column = 'VistaSType', edgecolor = 'whitesmoke', cmap = 'Set3', legend = True);

ImportError: The descartes package is required for plotting polygons in geopandas. You can install it using 'conda install -c conda-forge descartes' or 'pip install descartes'.

## Cows and CH$_4$
The data below come from the [USDA National Agricultural Statistics Service QuickStats](https://quickstats.nass.usda.gov/#D0FD8ACF-923C-3308-91BB-1E1ACE7662B9) platform. I downloaded the following data and included it in the cows.zip file:
* From the most recent California livestock census (2017), by California county:
    1. Total count of cattle including calves
    2. Total count of milk cows (a subset of (a))
    3. Total count of milk-producing operations that had sales in the census year

For the homework question, I just started with all cattle and used Appuhamy intake means to estimate per-county CH4 emissions. We can try to work on making this more granular later on. There should be some water quality regulatory filings that would include per-operation head counts, so maybe we can track that down.

In [12]:
nass = 'NASS'
cows_csv = 'NASS_2017_Census_CA_County_CattleInclCalves.csv'
ca_cows = pd.read_csv(os.sep.join(['.', nass, cows_csv]))

In [13]:
ca_cows.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,CENTRAL COAST,40,ALAMEDA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY","INVENTORY OF CATTLE, INCL CALVES","INVENTORY OF CATTLE, INCL CALVES: (1 TO 9 HEAD)",67,48.6
1,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,CENTRAL COAST,40,ALAMEDA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY","INVENTORY OF CATTLE, INCL CALVES","INVENTORY OF CATTLE, INCL CALVES: (10 TO 19 HEAD)",378,49.5
2,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,CENTRAL COAST,40,ALAMEDA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY","INVENTORY OF CATTLE, INCL CALVES","INVENTORY OF CATTLE, INCL CALVES: (100 TO 199 ...","2,293",30.1
3,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,CENTRAL COAST,40,ALAMEDA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY","INVENTORY OF CATTLE, INCL CALVES","INVENTORY OF CATTLE, INCL CALVES: (20 TO 49 HEAD)",851,33.7
4,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,CENTRAL COAST,40,ALAMEDA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY","INVENTORY OF CATTLE, INCL CALVES","INVENTORY OF CATTLE, INCL CALVES: (200 TO 499 ...","2,730",25.3


The NASS census bins head counts by size of operation/herd and also reports totals. Right now I only want totals.

In [14]:
ca_cows_totals = ca_cows[(ca_cows['Domain'] == 'TOTAL') & (ca_cows['Program'] == 'CENSUS')]
print(ca_cows_totals.shape)
ca_cows_totals.head()

(57, 21)


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
7,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,CENTRAL COAST,40,ALAMEDA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,NOT SPECIFIED,"18,524",5.6
15,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,SIERRA MOUNTAINS,60,ALPINE,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,NOT SPECIFIED,281,5.6
22,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,SIERRA MOUNTAINS,60,AMADOR,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,NOT SPECIFIED,"15,209",5.6
30,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,SACRAMENTO VALLEY,50,BUTTE,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,NOT SPECIFIED,"14,246",5.6
38,CENSUS,2017,END OF DEC,NaN,COUNTY,CALIFORNIA,6,SIERRA MOUNTAINS,60,CALAVERAS,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,NOT SPECIFIED,"22,093",5.6


In [15]:
ca_cows_totals.columns

Index(['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State',
       'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed',
       'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value',
       'CV (%)'],
      dtype='object')

In [16]:
# drop some unecessary columns
cows_ch4 = ca_cows_totals.drop(columns = ['Period', 'Week Ending', 'Ag District', 'Ag District Code', 'Zip Code', 'Region', 'watershed_code', 'Watershed', 'Domain Category'])

# 'Value' has counts of cattle but it's a series of unwieldy string objects with commas, so we want to turn those into integers
cows_ch4['Value'] = [int(i.replace(',', '')) for i in cows_ch4['Value']]

In [17]:
# define functions to compute EPA estimate of CH4 emissions by county
def ch4_epa(GEI, count):
    ch4 = count * (0.065 * GEI / 0.05565)
    return ch4

def ch4_appuhamy(DMI, count):
    ch4 = count * (9.6 + 22.1 * DMI)
    return ch4

In [18]:
# reference for intake values
intake

,DMI_mean,GEI_mean,dNDF_mean,EE_mean,Milk Fat,type
0,13.5,248.2,45.9,3.6,0.0,CA Heifer
1,9.9,181.2,47.9,3.2,0.0,CA Dry Cow
2,19.9,381.0,34.1,3.2,3.6,CA Lactating Cow
3,10.3,173.0,45.8,3.4,0.0,Generic estimate from literature


In [19]:
# use the generic estimates of GEI and DMI from literature
GEI_mean_generic = 173
DMI_mean_generic = 10.3

cows_ch4['CH4_epa'] = ch4_epa(GEI_mean_generic, cows_ch4['Value'])
cows_ch4['CH4_appuhamy'] = ch4_appuhamy(DMI_mean_generic, cows_ch4['Value'])

In [20]:
cows_ch4.head()

,Program,Year,Geo Level,State,State ANSI,County,County ANSI,Commodity,Data Item,Domain,Value,CV (%),CH4_epa,CH4_appuhamy
7,CENSUS,2017,COUNTY,CALIFORNIA,6,ALAMEDA,1,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,18524,5.6,3.743080e+06,4394448.52
15,CENSUS,2017,COUNTY,CALIFORNIA,6,ALPINE,3,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,281,5.6,5.678068e+04,66661.63
22,CENSUS,2017,COUNTY,CALIFORNIA,6,AMADOR,5,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,15209,5.6,3.073229e+06,3608031.07
30,CENSUS,2017,COUNTY,CALIFORNIA,6,BUTTE,7,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,14246,5.6,2.878639e+06,3379578.58
38,CENSUS,2017,COUNTY,CALIFORNIA,6,CALAVERAS,9,CATTLE,"CATTLE, INCL CALVES - INVENTORY",TOTAL,22093,5.6,4.464255e+06,5241122.39


In [21]:
# some checks that we're landing in about the right place with these estimates
pop = cows_ch4['Value']
epa = cows_ch4['CH4_epa']
appu = cows_ch4['CH4_appuhamy']

total_pop = pop.sum() # total population of cattle should be around 5M
ch4_epa_yr = (epa * 365 / 10**6).sum() # total CH4 emissions should be around 400K tons/yr
ch4_appu_yr = (appu * 365 / 10**6).sum() # total CH4 emissions should be around 400K tons/yr and higher than epa estimate

assert total_pop <= 6e6 and total_pop >= 4e6
assert ch4_epa_yr <= 5e5 and ch4_epa_yr >=3e5
assert ch4_appu_yr <= 5e5 and ch4_epa_yr >=3e5 and ch4_appu_yr > ch4_epa_yr

### Export CH4 estimates to CSV

In [22]:
cows_ch4.to_csv('cows_ch4.csv')

## Some stuff on a Jupyter-Drive workflow
This requires some backend work setting up the Drive API and OAuth credentials. [Some tips here](https://medium.com/@umdfirecoml/a-step-by-step-guide-on-how-to-download-your-google-drive-data-to-your-jupyter-notebook-using-the-52f4ce63c66c).


In [23]:
#!pip3 install df2gspread gspread
#!pip3 install pydrive
import gspread 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from oauth2client.service_account import ServiceAccountCredentials

ModuleNotFoundError: No module named 'gspread'

In [5]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('./jupyter-drive-294015-beb1b5310c70.json', scope) 
gc = gspread.authorize(credentials)

In [6]:
sheetkey = '1hvhs2L76VWbfcJPF0wGlM7iOIiCQh1I2ThQYZduNIsU'
book = gc.open_by_key(sheetkey) 
worksheet = book.worksheet("Sheet1") 
table = worksheet.get_all_values()

In [8]:
csvkey = '1HmVVn51T1W7JfHb5zIYb7wrtXMl04Env'

In [57]:
# 1. Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('./jupyter-drive-294015-beb1b5310c70.json', scope)
drive = GoogleDrive(gauth)
file_id = csvkey
downloaded = drive.CreateFile({'id': file_id}) # allows you to temporarily load your file in the notebook VM

# assume the file is called file.csv and it's located at the root of your drive
downloaded.GetContentFile('file.csv')
data = pd.read_csv('file.csv')

In [60]:
data

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,90210,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",SALES,"SALES: (50,000 OR MORE $)",2,NaN
1,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,90210,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",TOTAL,NOT SPECIFIED,2,NaN
2,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,90248,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",SALES,"SALES: (50,000 OR MORE $)",2,NaN
3,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,90248,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",TOTAL,NOT SPECIFIED,2,NaN
4,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,90265,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",TOTAL,NOT SPECIFIED,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,96115,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",TOTAL,NOT SPECIFIED,1,NaN
504,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,96121,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",TOTAL,NOT SPECIFIED,6,NaN
505,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,96128,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",TOTAL,NOT SPECIFIED,1,NaN
506,CENSUS,2007,YEAR,NaN,ZIP CODE,CALIFORNIA,6,NaN,NaN,NaN,...,99999,NaN,0,NaN,MILK,"MILK, INCL OTHER DAIRY PRODUCTS - OPERATIONS W...",SALES,"SALES: (50,000 OR MORE $)",11,NaN
